# Exports

In [ ]:
#export
TEST = 'test'

This is a test variable that will be available in our `./exports/` directory as `./exports/00_exports.ipynb` 

---

Save the notebook here before running the next cell.
Saving here writes the cells that have been run in this current session to file rather than a temporary file so that our `notebook2script.py` code can access all the `#export` cells that we want it to.

In [ ]:
# !python notebook2script.py 00_exports.ipynb
%run notebook2script.py 00_exports.ipynb

Converted 00_exports.ipynb to nb_00.py


### How it works:

Cells with the first line of the cell as: "`#export`" will be picked up by the `notebook2script.py` script and turned into a repository of *exported* or saved code that can be loaded and thus persist across the project development environment.

Jupyter notebooks are just json files where each cell is an entry in a list called `'cells'`. 
As seen below, each cell has a number of attributes: `cell_type`, `execution_unit`, `metadata`, `outputs`, and `source`.
The `notebook2script.py` script just finds all the cells that have `#export` as the first line in the cell and then it exports the rest of the code in those cells to a respective file in our `exports/` directory.

In [ ]:
import json
d = json.load(open('00_exports.ipynb', 'r'))['cells']
d[2]

{'cell_type': 'code',
 'execution_count': 1,
 'metadata': {},
 'outputs': [],
 'source': ['#export\n', "TEST = 'test'"]}

### Directory Layout

```bash
working-dir/
            00_exports.ipynb
            notebook2script.py
            exports/
                    nb_00.py
```

## Notebook to Script

The following is the `notebook2script` code in its entirety.

In [ ]:
import json, fire, re
from pathlib import Path

def is_export(cell):
    if cell['cell_type'] != 'code':
        return False
    src = cell['source']
    if len(src) == 0 or len(src[0]) < 7:
        return False
    # import pdb; pdb.set_trace()
    return re.match(pattern=r'^\s*#\s*export\s*$', string=src[0], flags=re.IGNORECASE) is not None

def notebook2script(fname):
    fname = Path(fname)
    fname_out = f'nb_{fname.stem.split("_")[0]}.py'
    main_dic = json.load(open(fname, 'r'))
    code_cells = [c for c in main_dic['cells'] if is_export(c)]
    module = f'''
#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# file to edit: dev_nb/{fname.name}
'''
    for cell in code_cells:
        module += ''.join(cell['source'][1:]) + '\n\n'
    # remove trailing spaces
    module = re.sub(pattern=r' +$', repl='', string=module, flags=re.MULTILINE)
    open(fname.parent/'exports'/fname_out, 'w').write(module[:-2])
    print(f"Converted {fname} to {fname_out}")

if __name__ == '__main__':
    fire.Fire(notebook2script)